# 여기부터 CNN모델 

In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
import numpy as np

# Initialising the CNN
# classifier = Sequential()
model = Sequential()

# Step 1 - Convolution
# classifier.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
# classifier.add(Conv2D(16, (3, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.25))

# classifier.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# classifier.add(Conv2D(64, (3, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.25))

# classifier.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
# classifier.add(Conv2D(32, (3, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))
# classifier.add(Dropout(0.25))

# classifier.add(Flatten())
# classifier.add(Dense(512, activation='relu'))
# classifier.add(Dropout(0.5))
# classifier.add(Dense(5, activation='softmax'))

model.add(Conv2D(16, (4, 4), padding='same', use_bias=False, input_shape=(64, 64, 1)))   # 사진 사이즈 조정
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))        # 앙상블 효과 

model.add(Conv2D(32, (4, 4), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
#model.add(Flatten())
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
#model.add(Flatten())
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.25))

#model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
#model.add(BatchNormalization(axis=3, scale=False))
#model.add(Activation("relu"))
model.add(Flatten())
#model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))     # 판단할 카테고리 수  9
model.summary()

# Compiling the CNN
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(rescale = 1./255)

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)   #스케일링을 줄여주는것 (normalization 적용해보면 더 좋아질 수 있음) 
                                   
                                   #validation_split=0.33)
                                   #otation_range=10,
                                   #width_shift_range=0.2,
                                   #height_shift_range=0.2,
                                   #shear_range=0.2)
                                   #zoom_range=[0.9, 2.2],
                                   #horizontal_flip=True,
                                   #vertical_flip=True,
                                   #fill_mode='nearest')
                                   #validation_split=0.33)

#test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/home/shy9546/Downloads/last_check/train',   # 훈련시킬 사진 경로
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (64, 64),    # 사이즈 지정해주기
                                                 batch_size = 15,
                                                 color_mode='grayscale',
                                                 #color_mode='rgb',
                                                 class_mode = 'categorical')
                                                 #subset="training")
validation_set = train_datagen.flow_from_directory('/home/shy9546/Downloads/last_check/test',
                                                 shuffle=True,
                                                 seed=13,
                                                 target_size = (64, 64),
                                                 batch_size = 10,
                                                 color_mode='grayscale',
                                                 #color_mode='rgb',
                                                 class_mode = 'categorical')
                                                 #subset="validation")



/home/shy9546/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        256       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 16)        48        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        8192      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        96        
__________

In [15]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('./log.csv', append=True, separator=';')

##hist = model.fit_generator(training_set,
#                         epochs = 50,   # 반복수 변경가능
#                         validation_data = validation_set
#                         )
STEP_SIZE_TRAIN = training_set.n
STEP_SIZE_VALID = validation_set.n
model.fit_generator(generator = training_set,
                    #steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validation_set,
                    #validation_steps=STEP_SIZE_VALID,
                   epochs=30)
from keras.models import load_model

model.save('cnn_attraction_keras_shape.h5')

# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)



Epoch 1/30
52/52 [==============================] - 8s 150ms/step - loss: 0.6920 - acc: 0.6038 - val_loss: 0.4786 - val_acc: 0.7562
Epoch 2/30
52/52 [==============================] - 7s 127ms/step - loss: 0.3392 - acc: 0.8501 - val_loss: 0.1277 - val_acc: 0.9562
Epoch 3/30
52/52 [==============================] - 7s 127ms/step - loss: 0.1672 - acc: 0.9320 - val_loss: 0.0356 - val_acc: 0.9875
Epoch 4/30
52/52 [==============================] - 7s 126ms/step - loss: 0.1135 - acc: 0.9577 - val_loss: 0.1980 - val_acc: 0.9125
Epoch 5/30
52/52 [==============================] - 6s 123ms/step - loss: 0.1015 - acc: 0.9655 - val_loss: 1.0973 - val_acc: 0.5875
Epoch 6/30
52/52 [==============================] - 6s 121ms/step - loss: 0.0753 - acc: 0.9718 - val_loss: 0.0347 - val_acc: 0.9937
Epoch 7/30
52/52 [==============================] - 7s 126ms/step - loss: 0.0461 - acc: 0.9859 - val_loss: 0.0169 - val_acc: 0.9937
Epoch 8/30
52/52 [==============================] - 6s 124ms/step - loss: 0.

In [ ]:
plt.plot()

In [4]:
prediction_set = train_datagen.flow_from_directory('/home/shy9546/Downloads/color_temp/test',
                                                  target_size=(64,64),
                                                  color_mode='grayscale',
                                                  #color_mode='rgb',
                                                  batch_size=1,
                                                  class_mode =None,
                                                  shuffle=False,
                                                  seed=13)

Found 10 images belonging to 1 classes.


In [5]:
from keras.models import load_model


#model.load('./cnn_attraction_keras_model11.h5')
#model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model = load_model('./cnn_attraction_keras_shape.h5')
# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            validation_set)
            #steps = 10)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
STEP_SIZE_TEST = prediction_set.n
prediction_set.reset()

output = model.predict_generator(
            prediction_set,
            verbose=1)

predicted_class_indices = np.argmax(output,axis=1)
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print(predictions)

print(output)
print(prediction_set.filenames)


-- Evaluate --
acc: 99.37%
-- Predict --
10/10 [==============================] - 0s 25ms/step
['elipse', 'elipse', 'elipse', 'elipse', 'elipse', 'circle', 'circle', 'elipse', 'elipse', 'elipse']
[[1.3740821e-10 1.0000000e+00]
 [1.2161780e-07 9.9999988e-01]
 [2.0282746e-06 9.9999797e-01]
 [4.2040793e-09 1.0000000e+00]
 [1.1627523e-10 1.0000000e+00]
 [9.3885255e-01 6.1147496e-02]
 [9.7461390e-01 2.5386054e-02]
 [5.7554310e-03 9.9424452e-01]
 [5.9771584e-03 9.9402279e-01]
 [3.3686626e-01 6.6313380e-01]]
['test_set/10.png', 'test_set/11.png', 'test_set/12.png', 'test_set/13.png', 'test_set/14.png', 'test_set/15.png', 'test_set/2.png', 'test_set/3.png', 'test_set/4.png', 'test_set/8.png']


In [ ]:
# model10 -> circle picture error